<a href="https://colab.research.google.com/github/amir-10/Projet_Big_Data/blob/main/Projet_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instalation de la bibliotheque java
! apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# instalation de spark
! wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

In [ ]:
# décompresser le dossier 
! tar xf /content/spark-3.3.1-bin-hadoop3.tgz


In [ ]:
# instalation de pyspark
! pip install -q findspark
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
# definir deux variables d'environement

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"  # l'endroit de java
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3" # l'endroit de spark
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.5 pyspark-shell' 
import findspark

findspark.init("spark-3.3.1-bin-hadoop3")


from pyspark import SparkContext, SparkConf

# lancement du spark en local (en une seule machine qui est la machine virtuelle de colab dans ce cas) avec 4 processus (workers node)
configuration = SparkConf().setAppName("name").setMaster("local[4]").set('spark.jars.packages', 'org.apache.spark:spark-avro_2.11:2.4.5')
# name: c'est le nom qu'on donne a notre app (code)
sc = SparkContext(conf=configuration) # l'objet 

In [ ]:
sc

<SparkContext master=local[4] appName=name>

In [ ]:
# L'objet sparkSession créé pour utiliser l'API Spark SQL
from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=configuration).getOrCreate()

In [ ]:
! wget -q http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz

In [ ]:
# décompresser le dossier 
! tar xf /content/20news-19997.tar.gz

In [ ]:
rdd1 = sc.wholeTextFiles("/content/20_newsgroups/alt.atheism")
rdd2 = sc.wholeTextFiles("/content/20_newsgroups/rec.sport.baseball")

In [ ]:
x=rdd2.take(2)

In [ ]:
def separateur(x):
  l=x[1].split("\n\n")
  return (x[0],l)

#separateur(x[1])

rdd1=rdd1.map(separateur)
rdd2=rdd2.map(separateur)






In [ ]:
# extraction des infos de l'entete 
test=rdd1.take(1)

In [ ]:
noor=test[0][1][0]
l=noor.split("\n")
d=dict()
for x in l:
  z=x.split(":")
  d[z[0]]=z[1]
print(d)

{'Newsgroups': ' alt.atheism', 'Path': ' cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!zaphod.mps.ohio-state.edu!cs.utexas.edu!newsfeed.rice.edu!rice!spac-at1-59.rice.edu!user', 'From': ' pww@spacsun.rice.edu (Peter Walker)', 'Subject': ' Re', 'Message-ID': ' <pww-210493002420@spac-at1-59.rice.edu>', 'Followup-To': ' alt.atheism,talk.religion.misc,talk.origins', 'Sender': ' news@rice.edu (News)', 'Organization': " I didn't do it, nobody saw me, you can't prove a thing.", 'References': ' <C5pr7q.56n@news.cso.uiuc.edu> <19APR93.14322063.0243@VM1.MCGILL.CA> <1qvmk2$csk@morrow.stanford.edu> <lt8d3bINNj1g@exodus.Eng.Sun.COM> <1993Apr20.154658@IASTATE.EDU>', 'Date': ' Wed, 21 Apr 1993 05', 'Lines': ' 24'}


In [ ]:
def extraction_informations_entete(entete):
  l=entete.split("\n")
  d=dict()
  for x in l:
    z=x.split(":")
    d[z[0]]=z[1]

  return d

In [ ]:
# extraire l'organisation du premier document du rdd1 (alt.atheism)
# par l'action first
doc1=rdd1.first()
entete=doc1[1][0]
# Dictionnaire des informations de l'entete
dict1_info=extraction_informations_entete(entete)
dict1_info

{'Newsgroups': ' alt.atheism',
 'Path': ' cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!zaphod.mps.ohio-state.edu!cs.utexas.edu!newsfeed.rice.edu!rice!spac-at1-59.rice.edu!user',
 'From': ' pww@spacsun.rice.edu (Peter Walker)',
 'Subject': ' Re',
 'Message-ID': ' <pww-210493002420@spac-at1-59.rice.edu>',
 'Followup-To': ' alt.atheism,talk.religion.misc,talk.origins',
 'Sender': ' news@rice.edu (News)',
 'Organization': " I didn't do it, nobody saw me, you can't prove a thing.",
 'References': ' <C5pr7q.56n@news.cso.uiuc.edu> <19APR93.14322063.0243@VM1.MCGILL.CA> <1qvmk2$csk@morrow.stanford.edu> <lt8d3bINNj1g@exodus.Eng.Sun.COM> <1993Apr20.154658@IASTATE.EDU>',
 'Date': ' Wed, 21 Apr 1993 05',
 'Lines': ' 24'}

In [ ]:
# selectionner Organization et Newsgroups
print(dict1_info['Newsgroups'])
print(dict1_info['Organization'])

 alt.atheism
 I didn't do it, nobody saw me, you can't prove a thing.


In [ ]:
# par l'action take
doc2=rdd2.take(2)
entete2=doc2[1][1][0]
# Dictionnaire des informations de l'entete
dict2_info=extraction_informations_entete(entete2)
dict2_info

{'Newsgroups': ' rec.sport.baseball',
 'Path': ' cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!gatech!usenet.ins.cwru.edu!agate!spool.mu.edu!uwm.edu!caen!uvaarpa!murdoch!delmarva.evsc.Virginia.EDU!drw3l',
 'From': ' drw3l@delmarva.evsc.Virginia.EDU (David Robert Walker)',
 'Subject': ' Re',
 'Message-ID': ' <C5L40C.9LC@murdoch.acc.Virginia.EDU>',
 'Sender': ' usenet@murdoch.acc.Virginia.EDU',
 'Organization': ' University of Virginia',
 'References': ' <1993Apr14.200649.12578@pts.mot.com> <8994@blue.cis.pitt.edu>',
 'Date': ' Fri, 16 Apr 1993 16',
 'Lines': ' 34'}

In [ ]:
# selectionner Organization et Newsgroups
print(dict2_info['Newsgroups'])
print(dict2_info['Organization'])
print(dict2_info['Subject'])
print(dict2_info['References'])

 rec.sport.baseball
 University of Virginia
 Re
 <1993Apr14.200649.12578@pts.mot.com> <8994@blue.cis.pitt.edu>


In [142]:
def extraction_informations_entete2(entete):
  if entete is not None:
    l=entete[1][0].split("\n")
    d=dict()
    if len(l)!=0:
      for x in l:
        z=x.split(":",1)
        if len(z)==2:
          d[z[0]]=z[1]
        if len(z)<2:
           pass
        

      return (entete[0],[d,entete[1][1]])

In [143]:
new_rdd1=rdd1.map(extraction_informations_entete2)
#rdd1.take(3)

In [145]:
new_rdd1.take(8)

[('file:/content/20_newsgroups/alt.atheism/53411',
  [{'Newsgroups': ' alt.atheism',
    'Path': ' cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!zaphod.mps.ohio-state.edu!cs.utexas.edu!newsfeed.rice.edu!rice!spac-at1-59.rice.edu!user',
    'From': ' pww@spacsun.rice.edu (Peter Walker)',
    'Subject': ' Re: The Universe and Black Holes, was Re: 2000 years.....',
    'Message-ID': ' <pww-210493002420@spac-at1-59.rice.edu>',
    'Followup-To': ' alt.atheism,talk.religion.misc,talk.origins',
    'Sender': ' news@rice.edu (News)',
    'Organization': " I didn't do it, nobody saw me, you can't prove a thing.",
    'References': ' <C5pr7q.56n@news.cso.uiuc.edu> <19APR93.14322063.0243@VM1.MCGILL.CA> <1qvmk2$csk@morrow.stanford.edu> <lt8d3bINNj1g@exodus.Eng.Sun.COM> <1993Apr20.154658@IASTATE.EDU>',
    'Date': ' Wed, 21 Apr 1993 05:28:18 GMT',
    'Lines': ' 24'},
   "In article <1993Apr20.154658@IASTATE.EDU>, kv07@IASTATE.EDU

In [ ]:
# Fusionner les deux RDD (union)
fusion=rdd1.union(rdd2)


In [ ]:
doc1=fusion.first()
entete=doc1[1][0]
# Dictionnaire des informations de l'entete
dict1_info=extraction_informations_entete(entete)
dict1_info

{'Newsgroups': ' alt.atheism',
 'Path': ' cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!zaphod.mps.ohio-state.edu!cs.utexas.edu!newsfeed.rice.edu!rice!spac-at1-59.rice.edu!user',
 'From': ' pww@spacsun.rice.edu (Peter Walker)',
 'Subject': ' Re',
 'Message-ID': ' <pww-210493002420@spac-at1-59.rice.edu>',
 'Followup-To': ' alt.atheism,talk.religion.misc,talk.origins',
 'Sender': ' news@rice.edu (News)',
 'Organization': " I didn't do it, nobody saw me, you can't prove a thing.",
 'References': ' <C5pr7q.56n@news.cso.uiuc.edu> <19APR93.14322063.0243@VM1.MCGILL.CA> <1qvmk2$csk@morrow.stanford.edu> <lt8d3bINNj1g@exodus.Eng.Sun.COM> <1993Apr20.154658@IASTATE.EDU>',
 'Date': ' Wed, 21 Apr 1993 05',
 'Lines': ' 24'}

In [ ]:
from pyspark.sql import Row
def ToRow(x):
  entete=x[1][0]
  #dict1_info=extraction_informations_entete(entete)
  row = Row(Path=x[0], 
            Contenu=x[1][1])
  return row

#fusion.take(1)
rddF=fusion.map(ToRow)


In [ ]:
rddF.take(5)

[Row(Path='file:/content/20_newsgroups/alt.atheism/53411', Contenu="In article <1993Apr20.154658@IASTATE.EDU>, kv07@IASTATE.EDU (Warren\nVonroeschlaub) wrote:\n> \n>   Let's say that we drop a marble into the black hole.  It races, ever faster,\n> towards the even horizon.  But, thanks to the curving of space caused by the\n> excessive gravity, as the object approaches the event horizon it has further to\n> travel.  Integrating the curve gives a time to reach the event horizon of . . \n> infinity.  So the math says that nothing can enter a black hole."),
 Row(Path='file:/content/20_newsgroups/alt.atheism/51172', Contenu='\njbrown@batman.bmd.trw.com writes:\n> > Sorry, but there are no supernatural\n> > forces necessary to create a pathogen.  You are saying, "Since\n> > diseases are bad, the bad entity must have created it."  So\n> > what would you say about acid rain, meteors falling from the\n> > sky, volcanoes, earthquakes, and other QUOTE UNQUOTE "Acts\n> > of God?" \n> \n> I would 

In [ ]:
#Create dataframe
df=spark.createDataFrame(rddF)
df.printSchema()
df.show()
#df.count()

root
 |-- Path: string (nullable = true)
 |-- Newsgroups: string (nullable = true)
 |-- Organization: string (nullable = true)
 |-- Contenu: string (nullable = true)



Py4JJavaError: ignored

In [ ]:
!pip install pandavro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import pandavro as pdx
filename = "df.avro"
df2=df.toPandas()
pdx.to_avro(filename, df2)

Py4JJavaError: ignored

In [ ]:
#lecture du fichier avro créé
saved = pdx.read_avro(filename)
print(saved)

                                                   Path  \
0         file:/content/20_newsgroups/alt.atheism/53411   
1         file:/content/20_newsgroups/alt.atheism/51172   
2         file:/content/20_newsgroups/alt.atheism/53803   
3         file:/content/20_newsgroups/alt.atheism/53298   
4         file:/content/20_newsgroups/alt.atheism/53116   
...                                                 ...   
1995  file:/content/20_newsgroups/rec.sport.baseball...   
1996  file:/content/20_newsgroups/rec.sport.baseball...   
1997  file:/content/20_newsgroups/rec.sport.baseball...   
1998  file:/content/20_newsgroups/rec.sport.baseball...   
1999  file:/content/20_newsgroups/rec.sport.baseball...   

                                                 Entete  \
0     Newsgroups: alt.atheism\nPath: cantaloupe.srv....   
1     Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....   
2     Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....   
3     Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...

In [ ]:
# sauvgarder le dataframe au format parquet
import pyarrow as pa
import pyarrow.parquet as pq
table = pa.Table.from_pandas(df2)
pq.write_table(table, 'df.parquet')

In [ ]:
# lecture du df a partir du fichier parquet
table2 = pq.read_table('df.parquet')
table2.to_pandas()

,Path,Entete,Contenu
0,file:/content/20_newsgroups/alt.atheism/53411,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,"In article <1993Apr20.154658@IASTATE.EDU>, kv0..."
1,file:/content/20_newsgroups/alt.atheism/51172,Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....,\njbrown@batman.bmd.trw.com writes:\n> > Sorry...
2,file:/content/20_newsgroups/alt.atheism/53803,Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....,sandvik@newton.apple.com (Kent Sandvik) writes:
3,file:/content/20_newsgroups/alt.atheism/53298,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,In article <C5pxqs.LM5@darkside.osrhe.uoknor.e...
4,file:/content/20_newsgroups/alt.atheism/53116,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,In article <C5Hr5M.KH2@ux1.cso.uiuc.edu> lis45...
...,...,...,...
1995,file:/content/20_newsgroups/rec.sport.baseball...,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,\n\t MLB Standings and Scores for Satruday...
1996,file:/content/20_newsgroups/rec.sport.baseball...,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,In article <1993Apr13.150904.25249@alchemy.che...
1997,file:/content/20_newsgroups/rec.sport.baseball...,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,Here is the OPI (Offensive Production Index) f...
1998,file:/content/20_newsgroups/rec.sport.baseball...,Path: cantaloupe.srv.cs.cmu.edu!rochester!udel...,In article <GRABINER.93Apr23112924@germain.har...


In [ ]:
df.select("Entete").show()

+--------------------+
|              Entete|
+--------------------+
|Newsgroups: alt.a...|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Xref: cantaloupe....|
|Newsgroups: alt.a...|
|Xref: cantaloupe....|
|Path: cantaloupe....|
|Newsgroups: alt.a...|
|Path: cantaloupe....|
|Newsgroups: alt.a...|
|Xref: cantaloupe....|
|Newsgroups: alt.a...|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Xref: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
|Path: cantaloupe....|
+--------------------+
only showing top 20 rows

